# Heat conduction and radiation verification

This is for verification of a new radiation model with temperature / wavelength dependent emissivity. See http://www.elmerfem.org/forum/viewtopic.php?p=27604

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Geometry, Mesh

In [2]:
from sphere_2D import mesh

heater_r_in = 0.4
heater_r_out = 0.5
insulation_r_in = 0.9
insulation_r_out = 1
mesh_size = 0.01

ph_heater, ph_insulation, ph_heater_in, ph_heater_out, ph_insulation_in, ph_insulation_out = mesh(heater_r_in, heater_r_out, insulation_r_in, insulation_r_out, mesh_size)

Screenshot of the mesh:

![geometry-mesh](geometry-mesh.png)

## Setup

The inner sphere is heated with a volumetric power of 30 kW.

There is surface-to-surface radiation between inner and outer sphere and inside of the inner one. At the outer sphere there is radiation to ambient with the ambient temperature $T_{amb}=300~\mathrm{K}$.

## Simulation (todo)

TODO: This needs to be adjusted to the new model! Alternatively, one could directly adjust the sif-file in the *simdata* subdirectory.

In [3]:
# from sphere_2D import elmer_setup
# from pyelmer.execute import run_elmer_grid, run_elmer_solver
# from pyelmer.post import scan_logfile

# elmer_setup(ph_heater, ph_insulation, ph_heater_in, ph_heater_out, ph_insulation_in, ph_insulation_out)
# run_elmer_grid("./simdata", "2d_sphere.msh")
# run_elmer_solver("./simdata")
# warn, err, stats = scan_logfile("./simdata")
# print("Warnings:", warn)
# print("Errors:", err)
# print("Statistics:", stats)

## Analytical solution
The analytical solution is computed according to [K. Dadzis, Modeling of directional solidification of multicrystalline silicon in a
traveling magnetic field, Dissertation, TU Bergakademie Freiberg, 2012, Online:
http://nbn-resolving.de/urn:nbn:de:bsz:105-qucosa-117492] and [H.D. Baehr, K. Stephan, Wärme- und Stoffübertragung, 7th ed., Springer-Verlag, Berlin Heidelberg New York, 2010. https://doi.org/10.1007/978-3-642-10194-6, Chapter 5.5.3]. The base parameters are:

In [10]:
P = 30000  # input power
T_amb = 300  # ambient temperature

eps_emission_insulator = 0.4  # emissivity
eps_reflection_insulation = 0.5
eps_emission_heater = 0.8
eps_reflection_heater = 0.9
lmbd_i = 0.5  # heat conductivity
lmbd_h = 20
sgm_sb = 5.670374419e-8  # Stefan-Boltzmann constant

r_hi = heater_r_in  # radius
r_ho = heater_r_out
r_ii = insulation_r_in
r_io = insulation_r_out

The temperatures at the outer sphere, the insulation, is given by:
\begin{align}
    T_{i,o} &= \left[ \frac{P}{\sigma_{sb} \epsilon_{i,e} 4 \pi r^2_{i,o}}+ T_a^4 \right]^{1/4},\\
    T_i(r) &= T_{i,o} + \frac{P}{4 \pi \lambda_i}\left[ \frac{1}{r} - \frac{1}{r_{i,o}} \right]
\end{align}


In [11]:
T_io = (P/(sgm_sb*eps_emission_insulator*4*np.pi * r_io**2) + T_amb**4)**0.25
print("Insulation outside temperature:", T_io)

Insulation outside temperature: 580.2423949435341


In [12]:
def T_i(r):
    return T_io + P/(4*np.pi*lmbd_i)*(1/r - 1/r_io)
T_ii = T_i(r_ii)
print("Insulation inside temperature:", T_ii)

Insulation inside temperature: 1110.7588719165187


The temperature inner sphere (heater) outer surface is computed from the radiative heat exchange. The index 1 is used for the heater, 2 for the insulation.

In [13]:
from sympy import init_printing, symbols, Eq, solve, pprint, pi
import IPython.display as disp


a1, a2 = symbols("a1, a2")  # surface area
eps_e1, eps_e2 = symbols("epsilon_e1, epsilon_e2")  # emission for outgoing radiation
eps_r1, eps_r2 = symbols("epsilon_r1, epsilon_r2")  # emission for reflection / absorption
eps_1, eps_2 = symbols("epsilon_1, epsilon_2")  # emission for old case
sigma = symbols("sigma")  # Stefan-Boltzmann constant
t1, t2 = symbols("T_1, T_2")  # temperatures
r1, r2 = symbols("r1, r2")  # radiosities
p = symbols("P")  # power

# eq1 = Eq(a1 / (1-eps_1)* (sigma*t1**4 - r1), p)  # old case
eq1 = Eq(p, a1 / (1-eps_r1)* (sigma*eps_e1*t1**4 - eps_r1 *r1))  # new case
r1 = solve([eq1], [r1])[r1]

eq2 = Eq(a1*(r1 - r2), p)
r2 = solve([eq2], [r2])[r2]

# eq3 = Eq(a1 / (1-eps_1)* (sigma*t1**4 - r1), -a2 / (1-eps_2)* (sigma*t2**4 - r2))  # old case
eq3 = Eq(a1 / (1-eps_r1)* (sigma*eps_e1*t1**4 - eps_r1 *r1), -a2 / (1-eps_r2)* (sigma*eps_e2*t2**4 - eps_r2 *r2))  # new case

sol = solve([eq3], [t1])
disp.display(Eq(t1, sol[-1][0]))

t1_val = sol[-1][0].subs(
    [
        (eps_e1, eps_emission_heater),
        (eps_r1, eps_reflection_heater),
        (eps_e2, eps_emission_insulator),
        (eps_r2, eps_reflection_insulation),
        (sigma, sgm_sb),
        (p, P),
        (a1, 4*pi* r_ho**2),
        (a2, 4*pi* r_ii**2),
        (t2, T_ii)]
)
T_ho = t1_val.evalf()
print("Heater outside temperature:", T_ho)

Eq(T_1, (-P*epsilon_r1/(a2*epsilon_e1*sigma) + P*epsilon_r1/(a2*epsilon_e1*epsilon_r2*sigma) + P/(a1*epsilon_e1*sigma) + T_2**4*epsilon_e2*epsilon_r1/(epsilon_e1*epsilon_r2))**(1/4))

Heater outside temperature: 1131.47210025574


The temperature distribution of the heater is given by:
\begin{align}
    T_{h}(r) &= T_{h,o} + \frac{P}{V_h}\frac{1}{3\lambda_{h}} \left[\frac{r^2_{h,o}}{2} - \frac{r^2}{2} + \frac{r^3_{h,i}}{r_{h,o}}-\frac{r^3_{h,i}}{r} \right],
\end{align}

In [14]:
V_h = 4/3 * np.pi * (r_ho**3 - r_hi**3)
def T_h(r):
    return T_ho + P/(V_h*3*lmbd_h)*(r_ho**2/2 - r**2/2 + r_hi**3/r_ho - r_hi**3/r)
print(T_h(r_hi))

1156.91080017617


## Comparison (todo)

Manual evaluation of numerically computed heat flux using ParaView, PlotOverLine, Save Data in csv format (extract-data.psvm).

In [9]:
# # analytical
# r_i = np.linspace(r_ii, r_io, 100)
# r_h = np.linspace(r_hi, r_ho, 100)

# # numerical
# df = pd.read_csv('./simdata/line-data.csv')
# fig, ax = plt.subplots(1, 1, figsize=(5.5, 4))
# ax.grid(linestyle=":")
# l1, = ax.plot(r_i, T_i(r_i), color="#1f77b4")
# l2, = ax.plot(r_h, T_h(r_h), color="#1f77b4")

# l3, = ax.plot(df['Points:0'], df['temperature'], color="#ff7f0e")

# ax.legend([l1, l3], ["analytical", "numerical"])
# # ax.legend()
# # ax.set_xlabel('radius [m]')
# # ax.set_ylabel('joule heat $\\left[\\frac{\\mathrm{MW}}{\mathrm{m}^3}\\right]$')
# # fig.tight_layout()
# # fig.savefig("verification_joule-heat.png")
# # plt.show()